### Введение
Разбиваем объекты на категории не по количеству, а по качеству. <br>
Наборы категорий без внутреннего упорядочения называются номинальными, например: <br>
- синий, красный, зеленый
- мужчина женщина
- банан, клубника, яблоко

С другой стороны, когда набор категорий имеет некое естественное упорядочение, его называют порядковым, например: <br>
- низкий, средний, высокий
- молодые, старые
- согласен, не согласен

Алгоритм k ближайших соседей предоставляет простой пример. Одним из шагов в алгоритме является вычисление расстояний между наблюдениями - часто с использованием евклидова расстояния:
$$
\sqrt{\sum \limits _{i=1} ^n (x_i-y_i)^2}
$$

Где x и y - это два наблюдения; i - номер признака наблюдений

Дан признак с номинальными классами, который не имеет внутреннй упорядоченности (например, яблоко, груша, банан)

In [25]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
feature = np.array([["Texas"],
                    ["California"],
                    ["Texas"],
                    ["Delaware"],
                    ["Texas"]])
one_hot = LabelBinarizer() # Сортировка в алфавитном порядке
one_hot.fit_transform(feature)

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

In [26]:
one_hot.classes_

array(['California', 'Delaware', 'Texas'], dtype='<U10')

In [27]:
one_hot.inverse_transform(one_hot.transform(feature))

array(['Texas', 'California', 'Texas', 'Delaware', 'Texas'], dtype='<U10')

In [28]:
import pandas as pd
pd.get_dummies(feature[:,0])

,California,Delaware,Texas
0,False,False,True
1,True,False,False
2,False,False,True
3,False,True,False
4,False,False,True


In [29]:
# Мультиклассовый признак
multiclass_feature = [("Texas", "Florida"),
                      ("California", "Alabama"),
                      ("Texas", "Florida"),
                      ("Delaware", "Florida"),
                      ("Texas", "Alabama")]
one_hot_multiclass = MultiLabelBinarizer()
one_hot_multiclass.fit_transform(multiclass_feature)

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

In [30]:
one_hot_multiclass.classes_

array(['Alabama', 'California', 'Delaware', 'Florida', 'Texas'],
      dtype=object)

В книге советуют: после кодирования признака в кодировку с одним активным состоянием часто рекомендуется отбрасывать один из закодированных в результирующей матрице признаков, чтобы избежать линейной зависимости. <br> <br>
Пока не понимаю о чем идет речь. Скорее всего в нашей таблице идет перевод категорий в значения 0 либо 1. А в линейных уравнениях это может вызывать проблемы

In [31]:
# Дан порядковый категориальный признак (высокий, средний, низкий)
dataframe = pd.DataFrame({"оценка": ["низкая", "низкая", "средняя", "средняя", "высокая"] })
scale_mapper = {"низкая":1, "средняя":2, "высокая":3}
dataframe["оценка"].map(scale_mapper)

0    1
1    1
2    2
3    2
4    3
Name: оценка, dtype: int64

In [32]:
from sklearn.feature_extraction import DictVectorizer
# Создать словарь
data_dict = [{"красный": 2, "синий": 4},
             {"красный": 4, "синий": 3},
             {"красный": 1, "желтый": 2},
             {"красный": 2, "желтый": 2}]
dictvectorizer = DictVectorizer(sparse=False)
feature = dictvectorizer.fit_transform(data_dict)
feature

array([[0., 2., 4.],
       [0., 4., 3.],
       [2., 1., 0.],
       [2., 2., 0.]])

In [33]:
features_names = dictvectorizer.get_feature_names_out()
pd.DataFrame(feature, columns=features_names)

,желтый,красный,синий
0,0.0,2.0,4.0
1,0.0,4.0,3.0
2,2.0,1.0,0.0
3,2.0,2.0,0.0


In [34]:
doc_l_word_count = {"красный": 2, "синий": 4}
doc_2_word_count = {"красный": 4, "синий": 3}
doc_3_word_count = {"красный": 1, "желтый": 2}
doc_4_word_count = {"красный": 2, "желтый": 2}
# Создать список
doc_word_counts = [doc_l_word_count, doc_2_word_count, doc_3_word_count, doc_4_word_count]
dictvectorizer.fit_transform (doc_word_counts)

array([[0., 2., 4.],
       [0., 4., 3.],
       [2., 1., 0.],
       [2., 2., 0.]])

Дан категориальный признак содержащий пропущенные значения, которые требуется заменить предсказанными значениями

In [43]:
from sklearn.neighbors import KNeighborsClassifier

X = np.array([[0, 2.10, 1.45],
              [1, 1.18, 1.33],
              [0, 1.22, 1.27],
              [1, -0.21, -1.19]])

X_with_nan = np.array([[np.nan, 0.87, 1.31],
                       [np.nan, -0.67, -0.22]])

# Обучаем модель: признаки = столбцы 1 и 2, метки = столбец 0
сlf = KNeighborsClassifier(n_neighbors=3, weights='distance')
trained_model = сlf.fit(X[:, 1:], X[:, 0])

# ПРЕДСКАЗЫВАЕМ МЕТКИ для новых ОБЪЕКТОВ (используем ТОЛЬКО признаки!)
new_features = X_with_nan[:, 1:]  # ← ключевое исправление!
imputed_values = trained_model.predict(new_features)

# Собираем полные строки: предсказанные метки + признаки
X_with_imputed = np.hstack((imputed_values.reshape(-1, 1), new_features))

# Объединяем с исходными данными
np.vstack((X_with_imputed, X))

array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

### Работа с несбалансирванными классами

Дан вектор целей с очень несбалансированными классами. Решение: собрать больше данных. <br>Если невозможно, то изменить метрические показатели, используемые для оценивания модели. <br>


In [44]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

iris = load_iris()
features = iris.data
target = iris.target
features = features[40:,:]
target = target[40:]
target = np.where((target == 0), 0, 1)
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [46]:
weights = {0: .9, 1: 0.1}
RandomForestClassifier(class_weight= weights)
RandomForestClassifier(bootstrap=True, class_weight= weights,
                       criterion= 'gini', max_depth= None,
                       max_features= 'auto', max_leaf_nodes = None,
                       min_impurity_decrease= 0.0, min_samples_leaf= 1,
                       min_samples_split = 2,
                       min_weight_fraction_leaf= 0.0, n_estimators = 10,
                       n_jobs = 1, oob_score= False, random_state= None,
                       verbose=0, warm_start=False)
i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]
n_class0 = len(i_class0)
n_class1 = len(i_class1)
i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)
np.hstack((target[i_class0], target[i_class1_downsampled]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [47]:
np.vstack((features[i_class0,:], features[i_class1_downsampled,:]))[0:5]

array([[5. , 3.5, 1.3, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4]])